In [1]:
import os
import cv2
import argparse
import numpy as np
import json
from skimage import color
import openslide
import pandas as pd
import glob
import csv
import shutil

In [17]:
file = '2_6928_A_0038465.png'
binary_contour =cv2.imread(file,cv2.IMREAD_GRAYSCALE)
binary_mask = cv2.threshold(binary_contour, 127, 255, cv2.THRESH_BINARY)[1]
contours, hierarchy  = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [44]:
geojson_features = []
contours, hierarchy  = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Loop through the contours
for i, contour in enumerate(contours):
    if len(contour) > 2:
        # Check if the contour has a parent (has hole(s) or not)
        if hierarchy[0][i][3] == -1:
            print(f'Contour {i} - NO parent')
            contour_json = []
            # Extract and format contour coordinates
            contour_coords = np.squeeze(contour).tolist()
            contour_coords.append(contour_coords[0])
            contour_coords = [[32 * c[0], 32 * c[1]] for c in contour_coords]
            contour_coords.append(contour_coords[0])
            contour_json.append(contour_coords)
            
            # Check for holes within the contour
            for j, child_hierarchy in enumerate(hierarchy[0]):
                if child_hierarchy[3] == i:
                    print(f'Contour {j} - is the hole of Contour {i}')
                    contour_coords = np.squeeze(contours[j]).tolist()
                    contour_coords.append(contour_coords[0])
                    contour_coords = [[32 * c[0], 32 * c[1]] for c in contour_coords]
                    contour_coords.append(contour_coords[0])
                    contour_json.append(contour_coords)
                    
            geojson_feature = {
                                    "type": "Feature",
                                    "geometry": {
                                        "type": "Polygon",
                                        "coordinates": contour_json
                                    },
                                    "properties": {
                                        "objectType": "annotation",
                                        "classification": {
                                            "name": "Cortex_QCed_new",
                                            "color": [29, 9, 7]
                                        }
                                    }
                                }
            geojson_features.append(geojson_feature)
with open(f'test.json', 'w') as f:
    json.dump(geojson_features, f)

Contour 1 - NO parent
Contour 2 - NO parent
Contour 3 - is the hole of Contour 2
Contour 4 - NO parent
Contour 5 - is the hole of Contour 4
Contour 6 - is the hole of Contour 4
